In [ ]:
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import pandas as pd 
import gc
from tqdm import tqdm
import tensorflow as tf
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

In [ ]:
LIMIT = 5.0
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
print('TensorFlow Limit {}GB'.format(LIMIT))
print('RAPIDS Limit {}GB'.format(16-LIMIT))

In [ ]:
test=pd.read_csv(r'../input/shopee-product-matching/test.csv')
if len(test)<=3:
    train=pd.read_csv(r'../input/shopee-product-matching/train.csv')
    image_paths='../input/shopee-product-matching/train_images/'+train.image
else:
    train=pd.read_csv(r'../input/shopee-product-matching/test.csv')
    image_paths='../input/shopee-product-matching/test_images/'+train.image

train.shape

In [ ]:
del test
gc.collect()

In [ ]:
resnet50_embedder=tf.keras.models.load_model('../input/shopee-arcface-trained-image-embedders/ResNet50_embedder_40_epochs.h5')

In [ ]:
def read_image(image):
    image = tf.io.read_file(image)
    image= tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize(image,(512,512))
    image = tf.cast(image, tf.float32) / 255.0
    return image

def get_dataset(image):
    dataset = tf.data.Dataset.from_tensor_slices(image)
    dataset = dataset.map(read_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(32)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
image_embeddings_resnet50=[]
chunksize=4096
chunks=len(train)//chunksize if len(train)%chunksize==0 else (len(train)//chunksize)+1
for chunk in tqdm(range(chunks)):
    start=chunk*chunksize
    end=min(len(train),start+chunksize)
    images=get_dataset(image_paths[start:end])
    embeddings=resnet50_embedder.predict(images)
    image_embeddings_resnet50.append(embeddings)
image_embeddings_resnet50=np.concatenate(np.array(image_embeddings_resnet50))
image_embeddings_resnet50.shape

In [ ]:
del resnet50_embedder, image_paths
gc.collect()

In [ ]:
image_embeddings_resnet50=np.array([i/np.linalg.norm(i) for i in tqdm(image_embeddings_resnet50)])
image_embeddings_resnet50.shape

In [ ]:
embedded_train=torch.from_numpy(image_embeddings_resnet50)
embedded_train=embedded_train.cuda()

In [ ]:
del image_embeddings_resnet50
gc.collect()

In [ ]:
matches=[]
chunksize=1024
chunks=len(train)//chunksize if len(train)%chunksize==0 else (len(train)//chunksize)+1
for chunk in tqdm(range(chunks)):
    start=chunk*chunksize
    end=min(len(train),start+chunksize)
    cossim=torch.matmul(embedded_train,embedded_train[start:end].T).T
    cossim=cossim.data.cpu().numpy()
    for per_posting in cossim:
        indices=np.where(per_posting>=0.75)[0]
        match=train.iloc[indices].posting_id.values
        if len(match.tolist())>50:
            ind=np.where(per_posting>=sorted(per_posting)[-50])[0]
            match=train.iloc[ind].posting_id.values
        matches.append(' '.join(match.tolist()))

In [ ]:
train['matches']=matches
train.head()

In [ ]:
submission=train[['posting_id','matches']]
submission.head()

In [ ]:
del train
gc.collect()

In [ ]:
submission.to_csv('submission.csv',index=False)